## MLP Regression

In [1]:
import os, sys
sys.path.append(os.path.join(os.path.abspath(''), '../'))

from compete.MLP import (
    MLPNet,
    PerInstCountDataset
)
from toyDb.utils.spv import SpvContext
from dataset.DatasetBuilder import build_dataset
from dataset.MapDataset import MapDataset, MapMixin
import transformers
import numpy as np
from transformers import Trainer, TrainingArguments
from transformers.trainer_callback import TrainerCallback
from misc.normalization import (
  NormalizerBase,
  Normalizer,
  DummyNormalizer,
  LogPlusNormalizer,
  LogNormalizer
)
from misc.metric import compute_metrics
import torch
from copy import deepcopy


In [2]:
hiddenDim = 128

grammar = SpvContext.getDefaultGrammar()
featureDim = len(grammar.instFormatsByOpcode)

model = MLPNet(featureDim, hiddenDim)
label_normalizer = LogNormalizer()

class TrainingSetMetricsReportCallback(TrainerCallback):
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_step_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy


class MLPPostProcessor(MapMixin):
    def __init__(self, targetNormalizer):
        # Tokenizer should support trace things
        self.targetNormalizer = targetNormalizer

    def doMap(self, elem):
        # labels
        elem[1] = self.targetNormalizer.normalize(elem[1])

        return elem

trainDatasetOrig = build_dataset('FragmentPerformanceTracedSnapshotDataset4096-3060', 'train')
testDatasetOrig = build_dataset('FragmentPerformanceTracedSnapshotDataset4096-3060', 'test')
trainDataset = PerInstCountDataset(grammar, trainDatasetOrig, True)
testDataset = PerInstCountDataset(grammar, testDatasetOrig, True)
trainDatasetMapped = MapDataset(
    trainDataset, MLPPostProcessor, [label_normalizer]
)
testDatasetMapped = MapDataset(
    testDataset, MLPPostProcessor, [label_normalizer]
)

def compute_fragment_performance_metrics(pred: 'transformers.trainer_utils.EvalPrediction'):
    """pred: inputs, label_ids and predictions, each of them is np.ndarray"""
    metrics = {}

    num_samples = pred.label_ids.shape[0]
    assert(pred.label_ids.shape == (num_samples,))
    assert(pred.predictions.shape == (num_samples, 1) or pred.predictions.shape == (num_samples, ))

    labels = pred.label_ids
    preds = pred.predictions.squeeze()
    # MSE
    metrics["mse_sqrt_raw"] = np.sqrt(((labels - preds)**2).mean(axis=0))
    # MAE
    metrics["mae_raw"] = np.mean(np.abs((labels - preds)))
    # MAPE
    metrics["mape_raw"] = np.mean(np.abs((labels - preds)/labels))
    # original (in time sense)
    orig_labels = label_normalizer.invNormalize(labels)
    orig_predictions = label_normalizer.invNormalize(pred.predictions.reshape((num_samples,)))
    metrics_merged = {
        **compute_metrics(orig_predictions, orig_labels),
        **metrics
    }

    return metrics_merged

# Normalization!

training_args = TrainingArguments(
    output_dir=f"compete_MLP_{hiddenDim}",
    num_train_epochs=50,
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    logging_dir=f"compete_MLP_{hiddenDim}",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_mape",
    greater_is_better=False,
    evaluation_strategy="steps",
    save_steps=500,
    learning_rate=5e-4,
    # torch_compile=True,
    save_total_limit=5
)

def collate_fn(batch):
    # print(batch)
    features = [item[0] for item in batch]
    targets = [item[1] for item in batch]
    return {
        "x": torch.tensor(features, dtype=torch.float32),
        "labels": torch.tensor(targets, dtype=torch.float32)
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainDatasetMapped,
    eval_dataset=testDatasetMapped,
    data_collator=collate_fn,
    compute_metrics=compute_fragment_performance_metrics
)

trainer.train()

/home/libreliu/Projects/NGPP/vkPredict/venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/900 [00:00<?, ?it/s]

{'loss': 81.2793, 'learning_rate': 0.0004944444444444445, 'epoch': 0.56}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 74.26960754394531, 'eval_mse_sqrt': 0.4326850175857544, 'eval_mae': 0.43264713883399963, 'eval_mape': 11906.0185546875, 'eval_mse_sqrt_ge_10000fps': 0.4335895776748657, 'eval_mae_ge_10000fps': 0.43358948826789856, 'eval_mape_ge_10000fps': 18802.154296875, 'eval_mse_sqrt_ge_1000_le_10000fps': 0.43318164348602295, 'eval_mae_ge_1000_le_10000fps': 0.433181494474411, 'eval_mape_ge_1000_le_10000fps': 1669.4796142578125, 'eval_mse_sqrt_ge_100_le_1000fps': 0.43058785796165466, 'eval_mae_ge_100_le_1000fps': 0.4305835962295532, 'eval_mape_ge_100_le_1000fps': 203.02809143066406, 'eval_mse_sqrt_ge_10_le_100fps': 0.41300877928733826, 'eval_mae_ge_10_le_100fps': 0.41283002495765686, 'eval_mape_ge_10_le_100fps': 25.50628089904785, 'eval_mse_sqrt_le_10fps': 0.20915623009204865, 'eval_mae_le_10fps': 0.20915623009204865, 'eval_mape_le_10fps': 0.9349676966667175, 'eval_mse_sqrt_raw': 8.617981910705566, 'eval_mae_raw': 8.406179428100586, 'eval_mape_raw': 0.904082179069519, 'eval_runtime': 10

...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.305363893508911, 'eval_mse_sqrt': 0.00572875514626503, 'eval_mae': 0.0008922209963202477, 'eval_mape': 1.7413427829742432, 'eval_mse_sqrt_ge_10000fps': 0.00013153112377040088, 'eval_mae_ge_10000fps': 7.315886614378542e-05, 'eval_mape_ge_10000fps': 2.378009557723999, 'eval_mse_sqrt_ge_1000_le_10000fps': 0.00030383685952983797, 'eval_mae_ge_1000_le_10000fps': 0.0002309653500560671, 'eval_mape_ge_1000_le_10000fps': 0.6544748544692993, 'eval_mse_sqrt_ge_100_le_1000fps': 0.0032626776956021786, 'eval_mae_ge_100_le_1000fps': 0.0026659949216991663, 'eval_mape_ge_100_le_1000fps': 0.8688464760780334, 'eval_mse_sqrt_ge_10_le_100fps': 0.023748451843857765, 'eval_mae_ge_10_le_100fps': 0.02037120796740055, 'eval_mape_ge_10_le_100fps': 0.9806228876113892, 'eval_mse_sqrt_le_10fps': 0.22322413325309753, 'eval_mae_le_10fps': 0.22322413325309753, 'eval_mape_le_10fps': 0.9978538751602173, 'eval_mse_sqrt_raw': 1.5183424949645996, 'eval_mae_raw': 1.1465376615524292, 'eval_mape_raw': 0.153411

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 2.3052539825439453, 'eval_mse_sqrt': 0.005728800781071186, 'eval_mae': 0.0008921999833546579, 'eval_mape': 1.7388591766357422, 'eval_mse_sqrt_ge_10000fps': 0.00013135382323525846, 'eval_mae_ge_10000fps': 7.30414321878925e-05, 'eval_mape_ge_10000fps': 2.3740787506103516, 'eval_mse_sqrt_ge_1000_le_10000fps': 0.0003038958238903433, 'eval_mae_ge_1000_le_10000fps': 0.00023097105440683663, 'eval_mape_ge_1000_le_10000fps': 0.6540383696556091, 'eval_mse_sqrt_ge_100_le_1000fps': 0.0032629426568746567, 'eval_mae_ge_100_le_1000fps': 0.002666317392140627, 'eval_mape_ge_100_le_1000fps': 0.8689952492713928, 'eval_mse_sqrt_ge_10_le_100fps': 0.02374872751533985, 'eval_mae_ge_10_le_100fps': 0.02037154696881771, 'eval_mape_ge_10_le_100fps': 0.9806439280509949, 'eval_mse_sqrt_le_10fps': 0.22322463989257812, 'eval_mae_le_10fps': 0.22322463989257812, 'eval_mape_le_10fps': 0.9978561401367188, 'eval_mse_sqrt_raw': 1.5183062553405762, 'eval_mae_raw': 1.1462132930755615, 'eval_mape_raw': 0.153399

TrainOutput(global_step=900, training_loss=7.090516800350613, metrics={'train_runtime': 2998.5178, 'train_samples_per_second': 150.391, 'train_steps_per_second': 0.3, 'train_loss': 7.090516800350613, 'epoch': 50.0})